In [10]:
import pandas as pd
import datetime

In [2]:
df = pd.read_csv('parsed2.csv',header=None, names = range(18), nrows = 11000000)

C:\Users\Hurle\AppData\Local\Temp\ipykernel_20948\1427644825.py:1: DtypeWarning: Columns (4,6,8,9,10,11,12,13,14,15,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('parsed2.csv',header=None, names = range(18), nrows = 11000000)


In [3]:
df[df[0] == "S"]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,S,0,0,10970642174571,O,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
237169,S,0,0,14400000186600,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10411212,S,0,0,34200000075128,Q,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
sys_df = df[df[0] == "S"]
sys_df = sys_df[[0,1,2,3,4]]
sys_df.columns = ["Code","num1","num2","ts","sys_code"]

In [7]:
stock_df = df[df[0] == 'R']
stock_df = stock_df.reset_index(drop=True)
stock_df.columns = ["Message_type","Stock_locate","Tracking_num","Ts","Stock","Market_cat","Financial_status","Round_lot_size","Round_lots_only","issue_class","issue_subtype","authenticity","SSTI","IPO","LULD","ETP","Lev","Inv"]

In [8]:
stock_df

,Message_type,Stock_locate,Tracking_num,Ts,Stock,Market_cat,Financial_status,Round_lot_size,Round_lots_only,issue_class,issue_subtype,authenticity,SSTI,IPO,LULD,ETP,Lev,Inv
0,R,1,0,11240690615180,A,N,,100,N,C,Z,P,N,,1,N,0.0,N
1,R,2,0,11240690740789,AA,N,,100,N,C,Z,P,N,,1,N,1.0,N
2,R,3,0,11240690786821,AAAU,P,,100,N,Q,I,P,N,,2,Y,1.0,N
3,R,4,0,11240690840751,AABA,Q,N,100,N,C,Q,P,N,N,2,N,1.0,N
4,R,5,0,11240690884618,AAC,N,,100,N,C,Z,P,N,,2,N,0.0,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8844,R,8845,0,11240925383765,ZXYZ.A,Q,N,100,N,C,Z,T,N,N,2,N,1.0,N
8845,R,8846,0,11240925418549,ZXZZT,G,N,100,N,W,Z,T,N,N,2,N,1.0,N
8846,R,8847,0,11240925445584,ZYME,N,,100,N,C,Z,P,N,,2,N,1.0,N
8847,R,8848,0,11240925470514,ZYNE,G,N,100,N,C,Z,P,N,N,2,N,0.0,N


In [11]:
#TS operations
stock_df["Ts_seconds"] = stock_df['Ts']/1e9
stock_df["Ts_stamp"] = stock_df["Ts_seconds"].apply(lambda x: datetime.datetime(2023,2,27) + datetime.timedelta(seconds = x))

In [12]:
#Stock_name operations
stock_df["Stock_normalized"] = stock_df["Stock"].apply(lambda x: x.split(" ")[0])

In [13]:
#Create Dictionary of stock locate ids
locate_dic = dict(zip(stock_df["Stock_locate"],stock_df["Stock_normalized"]))
rev_locate = {v: k for k,v in locate_dic.items()}

In [38]:
def normalize_series_name(sers):
    return sers.apply(lambda x: x.split(" ")[0])

def normalize_time(sers):
    return sers.apply(lambda x: datetime.datetime(2023,2,27) + datetime.timedelta(seconds = x/1e9))


In [22]:
exec_df = df[df[0] == 'E'][[0,1,2,3,4,5,6]].reset_index(drop=True)
exec_df.columns = ["Message_code","Locate","Track","Ts","Ref","Shares","Match"]

In [29]:
exec_df.columns = ["Message_code","Locate","Track","Ts","Ref","Shares","Match"]

In [31]:
exec_df["Stock_normalized"] = exec_df["Locate"].apply(lambda x: locate_dic[x].split(' ')[0])

In [42]:
exec_df["Ts_norm"] = normalize_time(exec_df[exec_df["Stock_normalized"] == 'AAPL']["Ts"])

In [46]:
aapl_df = exec_df[exec_df["Stock_normalized"] == 'AAPL'].reset_index(drop=True)

In [ ]:
aapl_adds = df[(df[0]=='A') & (df[7] == 'AAPL    ')]
aapl_adds = aapl_adds.iloc[:,0:9]
aapl_adds = aapl_adds.reset_index(drop=True)
aapl_adds.columns = ["Mess","Stock","Track","Ts","Order","BS","Shares","Stock_name","Price"]
aapl_adds["Stock_name"] = aapl_adds["Stock_name"].apply(lambda x: x.split(" ")[0])
aapl_adds["Price"] = aapl_adds["Price"].astype(float)

In [54]:
aapl_mpid_adds = df[(df[0]=='F') & (df[7] == 'AAPL    ')]

In [80]:
aapl_adds["Price"][0]

'1898600'

,0,1,2,3,4,5,6,7,8
237541,A,14,0,14400023941081,17613,B,53,AAPL,1898600
240092,A,14,0,14400756946890,52437,S,1,AAPL,2170500
240266,A,14,0,14400761460097,52669,B,1,AAPL,2052000
240593,A,14,0,14400769923694,53005,B,3,AAPL,2014800
240650,A,14,0,14400771392054,53069,S,2,AAPL,2280000
...,...,...,...,...,...,...,...,...,...
10964949,A,14,0,34201097623576,9919517,B,100.0,AAPL,2086100
10964965,A,14,0,34201097676372,9919537,B,5.0,AAPL,2086100
10973100,A,14,0,34201140584010,9927785,S,1200.0,AAPL,2086600
10995279,A,14,0,34201259074296,9948741,B,250.0,AAPL,2085200.0


In [59]:
rev_locate['AAPL']

14

In [61]:
aapl_cancels = df[(df[0]=='X') & (df[1] == 14)]

In [64]:
aapl_dels = df[(df[0]=='D') & (df[1] == 14)]

In [68]:
aapl_replaces = df[(df[0]=='U') & (df[1] == 14)]

0      2023-02-27 03:07:20.690615
1      2023-02-27 03:07:20.690741
2      2023-02-27 03:07:20.690787
3      2023-02-27 03:07:20.690841
4      2023-02-27 03:07:20.690885
                  ...            
8844   2023-02-27 03:07:20.925384
8845   2023-02-27 03:07:20.925419
8846   2023-02-27 03:07:20.925446
8847   2023-02-27 03:07:20.925471
8848   2023-02-27 03:07:20.925496
Name: Ts_seconds, Length: 8849, dtype: datetime64[ns]

TypeError: unsupported type for timedelta seconds component: Series

In [59]:
stock_df["Ts_seconds"] = datetime.datetime(2023,2,27) + datetime.timedelta(seconds = 11240)

0       11240.690615
1       11240.690741
2       11240.690787
3       11240.690841
4       11240.690885
            ...     
8844    11240.925384
8845    11240.925419
8846    11240.925446
8847    11240.925471
8848    11240.925496
Name: Ts_seconds, Length: 8849, dtype: float64

In [62]:
import datetime

In [63]:
td = datetime.timedelta(seconds = 11240)

In [81]:
datetime.datetime(2023,2,27) + td

datetime.datetime(2023, 2, 27, 3, 7, 20)

datetime.timedelta(days=1)